<span style="display:block;text-align:center;margin-right:105px"><img src="../media/logos/hacks-logo.png" width="200"/></span>

## Table of Contents

<ul>
    <li><a href='#Dependencies'>Dependencies</a></li>
    </li>
    <li><a href='#Modelling'>Modelling</a>
        <ol style='margin-top: 0em;'>
            <li><a href='#1.-State-Variables'>State Variables</a></li>
            <li><a href='#2.-System-Parameters'>System Parameters</a></li>
            <li><a href='#3.-Policy-Functions'>Policy Functions</a></li>
            <li><a href='#4.-State-Update-Functions'>State Update Functions</a></li>
            <li><a href='#5.-Partial-State-Update-Blocks'>Partial State Update Blocks</a></li>
        </ol>
    </li>
    <li><a href='#Simulation'>Simulation</a>
        <ol style='margin-top: 0em;' start="6">
            <li><a href='#6.-Configuration'>Configuration</a></li>
            <li><a href='#7.-Execution'>Execution</a></li>
            <li><a href='#8.-Output-Preparation'>Output Preparation</a></li>
            <li><a href='#9.-Analysis'>Analysis</a></li>
        </ol>
    </li>
</ul>

---

# Dependencies

In [1]:
# cadCAD standard dependencies

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

# Included with cadCAD
import pandas as pd

# Additional dependencies

# For analytics
import numpy as np
rng = np.random.default_rng(42)

# For visualization
import panel as pn
import plotly.express as px

# From our model
from parameterized.proposal_inverter import Wallet

# Modelling

* Have a pool of the agents, like a dictionairy
* Have a pool of proposals, like a dictionairy
* Track the connections between the agents and the PI

The wallets are initialized with random funds.   
Let's start the simulation with 1,000,000 Funds, 100 wallets, 0 owners, 0 proposals, 0 brokers, 0 payers.  
Scenario 1: Largest 20% of wallets become owners.    
Scenario 2: All wallets have an equal chance of becoming owners.

Some wallets will become owners  
1,000,000 Funds, 100 Wallets, 10 Owners, 10 Proposal, 0 Brokers, 0 Payers

Some Brokers Join Proposals and some Payers join proposals  
1,000,000 Funds, 100 Wallets, 10 Owners, 10 Proposal, 7 Brokers, 5 Payers

On every step of the simulation, as an agent (Wallet) I have three considerations to make (3-action space)
1. Become an Owner and start a new Proposal
2. For each proposal, consider if I want to become a payer
3. For each proposal, consider if I want to become a broker
4. If I am a broker, I can make a claim on the proposals that I am a broker on
5. If I am an owner, I can cancel a proposal

Note for implementation. Pools of agents and wallets to be 'global' outside of the simulation state, and referenced as we go. 

## 1. State Variables

The simulation maintains a constant number of funds in the economy and a constant number of wallets that act as agents. Each wallet begins with a random amount of funds that have been distributed from the total.

Over time, the agents will deploy, contribute to, or join proposals, and this will increase the number of owners, payers, and brokers, respectively.

### 1.1 Initial State

In [2]:
initial_state = {
    # Initial conditions of the system
    'total_funds': 1000000,
    'wallets': 100,
    'proposals': 0,
    
    # Distribution of agent actions
    'owners': 0,
    'payers': 0,
    'brokers': 0,
    
    # Proposal properties
    "funds_staked": 0,
    "funds_claimed": 0,
    "funds_contributed": 0,
}

### 1.2 Global Variables

In [3]:
# These are the global variables representing the pool of wallets and pool of proposals
WALLETS = dict()
PROPOSALS = dict()

funds_distribution = rng.random(size=100)
funds_distribution = funds_distribution / np.sum(funds_distribution) * initial_state["total_funds"]

for funds in funds_distribution:
    wallet = Wallet(funds=funds)
    WALLETS[wallet.public] = wallet

## 2. System Parameters

Each of the actions and the hyperparameters of each action can be represented as a function that samples from some probability distribution.

- For the actions, the function should return a boolean, indicating whether or not an agent will perform this action during the current epoch.
- For the hyperparameters, the functions should return the value of the hyperparameter.

### 2.1 Action Space Functions

Each action space function represents an agent's decision to perform an action. They should return `True` if the agent decides to perform the action, and `False` otherwise.

These functions are called in the policy functions, but kept separate for easy plug and play for different agent behaviours.

In [4]:
def a_static_one_percent(wallet: Wallet) -> bool:
    """Returns `True` with a probability of 1%."""
    return rng.random() < 0.01

### 2.2 Hyperparameter Functions

The hyperparameter functions represent the specifics of each action the agent may perform. For example, if the agent decides to join a proposal, then the agent must also decide how much to stake. A hypyerparameter function is then defined to decide how much to stake.

These functions are called in the policy functions, but kept separate for easy plug and play for different agent behaviours.

In [5]:
def h_join_proposal(wallet: Wallet):
    return rng.choice(list(PROPOSALS.keys()))

def h_claim_proposal(wallet: Wallet):
    return rng.choice(list(wallet.joined))

def h_leave_proposal(wallet: Wallet):
    return rng.choice(list(wallet.joined))

def h_join_stake(wallet: Wallet) -> float:
    return 10

def h_pay_proposal(wallet: Wallet):
    return rng.choice(list(PROPOSALS.keys()))

def h_pay_contribution(wallet: Wallet) -> float:
    return 50

def h_vote_proposal(wallet: Wallet):
    return rng.choice(list(wallet.paid))

def h_vote_broker(wallet: Wallet) -> int:
    return rng.choice(list(WALLETS.keys()))

def h_vote_result(wallet: Wallet) -> bool:
    return True

def h_deploy_initial_funds(wallet: Wallet) -> float:
    return 50

def h_cancel_proposal(wallet: Wallet):
    return rng.choice(list(wallet.owned))

### 2.3 System Parameters

In [6]:
system_params = {
    # Probabilities defining the action space
    "join": [a_static_one_percent],
    "claim": [a_static_one_percent],
    "leave": [a_static_one_percent],
    "pay": [a_static_one_percent],
    "vote": [a_static_one_percent],
    "deploy": [a_static_one_percent],
    "cancel": [a_static_one_percent],
    
    # The hyperparameters of each action
    "join_proposal": [h_join_proposal],
    "join_stake": [h_join_stake],
    
    "claim_proposal": [h_claim_proposal],
    
    "leave_proposal": [h_leave_proposal],

    "pay_proposal": [h_pay_proposal],
    "pay_contribution": [h_pay_contribution],
    
    "vote_proposal": [h_vote_proposal],
    "vote_broker": [h_vote_broker],
    "vote_result": [h_vote_result],
    
    "deploy_initial_funds": [h_deploy_initial_funds],
    
    "cancel_proposal": [h_cancel_proposal],
}

## 3. Policy Functions

The policy functions carry out the actions of each agent and will return a dictionary that contains a summary or aggregate function of the actions that have taken place during the current epoch.

In [7]:
def p_iter_epoch(params, substep, state_history, previous_state):
    for proposal in PROPOSALS.values():
        proposal.iter_epoch()
        
    return dict()


def p_join(params, substep, state_history, previous_state):
    n_joined = 0
    funds_staked = 0
    
    for wallet in WALLETS.values():
        if params["join"](wallet) and len(PROPOSALS) > 0:
            proposal = params["join_proposal"](wallet)
            stake = params["join_stake"](wallet)
            
            wallet = PROPOSALS[proposal].add_broker(wallet, stake)
            
            n_joined += 1
            funds_staked += stake
            
    return {
        "n_joined": n_joined,
        "funds_staked": funds_staked,
    }
            
    
def p_claim(params, substep, state_history, previous_state):
    n_claimed = 0
    funds_claimed = 0
    
    for wallet in WALLETS.values():
        if params["claim"](wallet) and len(wallet.joined) > 0:
            proposal = params["claim_proposal"](wallet)
            initial_funds = wallet.funds
            
            wallet = PROPOSALS[proposal].claim_broker_funds(wallet)
            
            n_claimed += 1
            funds_claimed += wallet.funds - initial_funds
            
    return {
        "n_claimed": n_claimed,
        "funds_claimed": funds_claimed,
    }


def p_leave(params, substep, state_history, previous_state):
    n_left = 0
    funds_claimed = 0
    
    for wallet in WALLETS.values():
        if params["leave"](wallet) and len(wallet.joined) > 0:
            proposal = params["leave_proposal"](wallet)
            initial_funds = wallet.funds
            
            wallet = PROPOSALS[proposal].remove_broker(wallet)
            wallet = PROPOSALS[proposal].claim_broker_funds(wallet)
            
            n_left += 1
            funds_claimed += wallet.funds - initial_funds
            
    return {
        "n_left": n_left,
        "funds_claimed": funds_claimed,
    }


def p_pay(params, substep, state_history, previous_state):
    n_paid = 0
    funds_contributed = 0

    for wallet in WALLETS.values():
        if params["pay"](wallet) and len(PROPOSALS) > 0:
            proposal = params["pay_proposal"](wallet)
            contribution = params["pay_contribution"](wallet)
            
            wallet = PROPOSALS[proposal].pay(wallet, contribution)
            
            n_paid += 1
            funds_contributed += contribution
    
    return {
        "n_paid": n_paid,
        "funds_contributed": funds_contributed
    }


def p_vote(params, substep, state_history, previous_state):
    n_voted = 0
    voted_yes = 0
    
    for wallet in WALLETS.values():
        if params["vote"](wallet) and len(wallet.paid) > 0:
            proposal = params["vote_proposal"](wallet)
            broker = WALLETS[params["vote_broker"](wallet)]
            result = params["vote_result"](wallet)
            
            PROPOSALS[proposal].vote_broker(wallet, broker, result)
            
            n_voted += 1
            voted_yes += result * 1
            
    return {
        "n_voted": n_voted,
        "voted_yes": voted_yes,
    }


def p_deploy(params, substep, state_history, previous_state):
    n_deployed = 0
    
    for wallet in WALLETS.values():
        if params["deploy"](wallet):
            initial_funds = params["deploy_initial_funds"](wallet)
            
            proposal = wallet.deploy(
                initial_funds=initial_funds,
            )
            
            if proposal is not None:
                PROPOSALS[proposal.public] = proposal
            
                n_deployed += 1
        
    return {"n_deployed": n_deployed}


def p_cancel(params, substep, state_history, previous_state):
    n_cancelled = 0
    n_left = 0
    
    for wallet in WALLETS.values():
        if params["cancel"](wallet) and len(wallet.owned) > 0:
            proposal = params["cancel_proposal"](wallet)
            
            wallet = PROPOSALS[proposal].cancel(wallet)
            
            brokers = list(PROPOSALS[proposal].broker_agreements.keys())
            for broker in brokers:
                WALLETS[broker] = PROPOSALS[proposal].remove_broker(WALLETS[broker])
                
                n_left += 1
                
            n_cancelled += 1
            
    return {
        "n_cancelled": n_cancelled,
        "n_left": n_left,
    }

## 4. State Update Functions

In [8]:
def s_proposals(params, substep, state_history, previous_state, policy_input):
    return "proposals", len(PROPOSALS)


def s_owners(params, substep, state_history, previous_state, policy_input):
    n_owners = previous_state["owners"] + policy_input["n_deployed"] - policy_input["n_cancelled"]
    
    return "owners", n_owners


def s_payers(params, substep, state_history, previous_state, policy_input):
    n_payers = previous_state["payers"] + policy_input["n_paid"]
    
    return "payers", n_payers


def s_brokers(params, substep, state_history, previous_state, policy_input):
    n_brokers = previous_state["brokers"] + policy_input["n_joined"] - policy_input["n_left"]
    
    return "brokers", n_brokers


def s_funds_staked(params, substep, state_history, previous_state, policy_input):
    funds_staked = previous_state["funds_staked"] + policy_input["funds_staked"]
    
    return "funds_staked", funds_staked


def s_funds_claimed(params, substep, state_history, previous_state, policy_input):
    funds_claimed = previous_state["funds_claimed"] + policy_input["funds_claimed"]
    
    return "funds_claimed", funds_claimed


def s_funds_contributed(params, substep, state_history, previous_state, policy_input):
    funds_contributed = previous_state["funds_contributed"] + policy_input["funds_contributed"]
    
    return "funds_contributed", funds_contributed

## 5. Partial State Update Blocks

In [9]:
partial_state_update_blocks = [
    # Go to next epoch
    {
        "policies": {
            "iter_epoch": p_iter_epoch,
        },
        "variables": {
        },
    },
    # Agents perform actions
    {   
        "policies": {
            "join": p_join,
            "claim": p_claim,
            "leave": p_leave,
            "pay": p_pay,
            "vote": p_vote,
            "deploy": p_deploy,
            "cancel": p_cancel,
        },
        'variables': {
            "proposals": s_proposals,
            
            # Distribution of agent actions
            "owners": s_owners,
            "payers": s_payers,
            "brokers": s_brokers,
            
            # Proposal properties
            "funds_staked": s_funds_staked,
            "funds_claimed": s_funds_claimed,
            "funds_contributed": s_funds_contributed,
        }
    }
]

# Simulation

## 6. Configuration

In [10]:
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation ("Monte Carlo runs")
    "T": range(400), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

In [11]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

## 7. Execution

In [12]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (400, 18, 1, 9)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal ha

Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled


Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled


Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Owner has insufficient funds to deploy proposal inverter
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been can

Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Failed to add broker, proposal has been cancelled
Total execution time: 2.21s


## 8. Output Preparation

In [13]:
# Convert raw results to a Pandas DataFrame
df = pd.DataFrame(raw_result)

What kind of metrics?
# Total Number of Agents
Can an agent of the simulation have multiple roles? For example, an owner who is also a broker, who is also a Payer?
# Number of Owners
# Number of Payers
# Number of Brokers
# Number of ProposalInverters

Can track the wallets of all of the 'agents'. And the wallets ProposalInverter Contracts. 

Probably want some sort of proxy of the system like "Production". Can also have an idea of "Productivity". 

Note: Whitelists are to be implemented, and consensus system for modifying whitelists. 

In [14]:
df

,total_funds,wallets,proposals,owners,payers,brokers,funds_staked,funds_claimed,funds_contributed,simulation,subset,run,substep,timestep
0,1000000,100,0,0,0,0,0,0,0,0,0,1,0,0
1,1000000,100,0,0,0,0,0,0,0,0,0,1,1,1
2,1000000,100,1,1,0,0,0,0,0,0,0,1,2,1
3,1000000,100,1,1,0,0,0,0,0,0,0,1,1,2
4,1000000,100,3,3,2,1,10,0,100,0,0,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,1000000,100,413,194,387,181,4000,0,19350,0,0,1,2,398
797,1000000,100,413,194,387,181,4000,0,19350,0,0,1,1,399
798,1000000,100,414,193,388,180,4010,0,19400,0,0,1,2,399
799,1000000,100,414,193,388,180,4010,0,19400,0,0,1,1,400


In [15]:


# Insert cadCAD parameters for each configuration into DataFrame
for config in configs:
    # Get parameters from configuration
    parameters = config.sim_config['M']
    # Get subset index from configuration
    subset_index = config.subset_id
    
    # For each parameter key value pair
    for (key, value) in parameters.items():
        # Select all DataFrame indices where subset == subset_index
        dataframe_indices = df.eval(f'subset == {subset_index}')
        # Assign each parameter key value pair to the DataFrame for the corresponding subset
        df.loc[dataframe_indices, key] = value

df.head(10)

,total_funds,wallets,proposals,owners,payers,brokers,funds_staked,funds_claimed,funds_contributed,simulation,...,join_stake,claim_proposal,leave_proposal,pay_proposal,pay_contribution,vote_proposal,vote_broker,vote_result,deploy_initial_funds,cancel_proposal
0,1000000,100,0,0,0,0,0,0,0,0,...,<function h_join_stake at 0x7f200c83adc0>,<function h_claim_proposal at 0x7f200c83aee0>,<function h_leave_proposal at 0x7f200c83a9d0>,<function h_pay_proposal at 0x7f200c83ae50>,<function h_pay_contribution at 0x7f200c5eb0d0>,<function h_vote_proposal at 0x7f200c5eb160>,<function h_vote_broker at 0x7f200c5eb1f0>,<function h_vote_result at 0x7f200c5eb280>,<function h_deploy_initial_funds at 0x7f200c5e...,<function h_cancel_proposal at 0x7f200c5eb3a0>
1,1000000,100,0,0,0,0,0,0,0,0,...,<function h_join_stake at 0x7f200c83adc0>,<function h_claim_proposal at 0x7f200c83aee0>,<function h_leave_proposal at 0x7f200c83a9d0>,<function h_pay_proposal at 0x7f200c83ae50>,<function h_pay_contribution at 0x7f200c5eb0d0>,<function h_vote_proposal at 0x7f200c5eb160>,<function h_vote_broker at 0x7f200c5eb1f0>,<function h_vote_result at 0x7f200c5eb280>,<function h_deploy_initial_funds at 0x7f200c5e...,<function h_cancel_proposal at 0x7f200c5eb3a0>
2,1000000,100,1,1,0,0,0,0,0,0,...,<function h_join_stake at 0x7f200c83adc0>,<function h_claim_proposal at 0x7f200c83aee0>,<function h_leave_proposal at 0x7f200c83a9d0>,<function h_pay_proposal at 0x7f200c83ae50>,<function h_pay_contribution at 0x7f200c5eb0d0>,<function h_vote_proposal at 0x7f200c5eb160>,<function h_vote_broker at 0x7f200c5eb1f0>,<function h_vote_result at 0x7f200c5eb280>,<function h_deploy_initial_funds at 0x7f200c5e...,<function h_cancel_proposal at 0x7f200c5eb3a0>
3,1000000,100,1,1,0,0,0,0,0,0,...,<function h_join_stake at 0x7f200c83adc0>,<function h_claim_proposal at 0x7f200c83aee0>,<function h_leave_proposal at 0x7f200c83a9d0>,<function h_pay_proposal at 0x7f200c83ae50>,<function h_pay_contribution at 0x7f200c5eb0d0>,<function h_vote_proposal at 0x7f200c5eb160>,<function h_vote_broker at 0x7f200c5eb1f0>,<function h_vote_result at 0x7f200c5eb280>,<function h_deploy_initial_funds at 0x7f200c5e...,<function h_cancel_proposal at 0x7f200c5eb3a0>
4,1000000,100,3,3,2,1,10,0,100,0,...,<function h_join_stake at 0x7f200c83adc0>,<function h_claim_proposal at 0x7f200c83aee0>,<function h_leave_proposal at 0x7f200c83a9d0>,<function h_pay_proposal at 0x7f200c83ae50>,<function h_pay_contribution at 0x7f200c5eb0d0>,<function h_vote_proposal at 0x7f200c5eb160>,<function h_vote_broker at 0x7f200c5eb1f0>,<function h_vote_result at 0x7f200c5eb280>,<function h_deploy_initial_funds at 0x7f200c5e...,<function h_cancel_proposal at 0x7f200c5eb3a0>
5,1000000,100,3,3,2,1,10,0,100,0,...,<function h_join_stake at 0x7f200c83adc0>,<function h_claim_proposal at 0x7f200c83aee0>,<function h_leave_proposal at 0x7f200c83a9d0>,<function h_pay_proposal at 0x7f200c83ae50>,<function h_pay_contribution at 0x7f200c5eb0d0>,<function h_vote_proposal at 0x7f200c5eb160>,<function h_vote_broker at 0x7f200c5eb1f0>,<function h_vote_result at 0x7f200c5eb280>,<function h_deploy_initial_funds at 0x7f200c5e...,<function h_cancel_proposal at 0x7f200c5eb3a0>
6,1000000,100,3,3,3,1,10,0,150,0,...,<function h_join_stake at 0x7f200c83adc0>,<function h_claim_proposal at 0x7f200c83aee0>,<function h_leave_proposal at 0x7f200c83a9d0>,<function h_pay_proposal at 0x7f200c83ae50>,<function h_pay_contribution at 0x7f200c5eb0d0>,<function h_vote_proposal at 0x7f200c5eb160>,<function h_vote_broker at 0x7f200c5eb1f0>,<function h_vote_result at 0x7f200c5eb280>,<function h_deploy_initial_funds at 0x7f200c5e...,<function h_cancel_proposal at 0x7f200c5eb3a0>
7,1000000,100,3,3,3,1,10,0,150,0,...,<function h_join_stake at 0x7f200c83adc0>,<function h_claim_proposal at 0x7f200c83aee0>,<function h_leave_proposal at 0x7f200c83a9d0>,<function h_pay_proposal at 0x7f200c83ae50>,<function h_pay_contribution at 0x7f200c5eb0d0>,<function h_vote_proposal at 0x7f200c5eb160>,<functio

## 9. Analysis

In [16]:
# Visualize how the predator and prey populations change over time

# Notice that the populations are more chaotic when the prey birth rate is higher,
# and the system is more stable when it is lower.

px.line(
    df,
    x='timestep', # Variable on the horizontal axis
    y=['predator_population', 'prey_population'], # Variables on the vertical axis
    line_group='run', # One line for each MC run
    facet_row='prey_birth_parameter', # Create a figure for each `prey_birth_parameter` parameter sweep
    log_y=True, # Use log scale on the vertical axis
    height=800,
)

ValueError: All arguments should have the same length. The length of argument `y` is 2, whereas the length of  previously-processed arguments ['timestep'] is 801

In [ ]:
# Visualize how the predator and prey populations compare

# Notice that they tend to have cycles around a fixed point
# which means that their populations are never static,
# but rather cyclic with ups and downs.

px.line(
    df,
    x='predator_population', # Variable on the horizontal axis
    y='prey_population', # Variable on the vertical axis
    color='run', # Color by MC run
    facet_row='prey_birth_parameter', # Create a figure for each `prey_birth_parameter` parameter sweep
    log_x=True, # Use log scale on the horizontal axis
    log_y=True, # Use log scale on the vertical axis
    height=800,
)